In [1]:
import subprocess
import numpy as np
import pandas as pd
import scanpy as sc
import random
import string
import subprocess
import anndata as an
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy 
import warnings
from collections import Counter
from google.cloud import storage
import re
import pyarrow.feather as feather
from scipy import sparse
from anndata import AnnData

### Process CPM file

In [ ]:
###direct download from website

###Meta data 
readRDS = robjects.r['readRDS']
meta = readRDS('CCLE_metadata.RDS') 
meta = pandas2ri.rpy2py_dataframe(meta)

####CPM data
df = readRDS('CCLE_scRNAseq_CPM.RDS')

####Gene locus
gene_locus = readRDS('gene_locus.RDS')
gene_locus_df = pandas2ri.rpy2py_dataframe(gene_locus)

##Copy number
copy_number = readRDS('CCLE_copynumber_bygene.RDS')
copy_number_df = pandas2ri.rpy2py_dataframe(copy_number)
copy_number_df = copy_number_df.T.reset_index()

In [ ]:
merge_copy = meta.reset_index().merge(copy_number_df, left_on = "index", right_on = "index" )

In [ ]:
###Merge gene names 

gene_name = pd.read_csv("/data/gene_name.txt", sep="\t", header = None)
gene_name_short = pd.read_csv("/data/gene_name_short.txt", sep="\t", header = None)
gene_name.index = gene_name[0]
gene_name_short.index = gene_name_short[0]
gene_name.columns = ['gene']
gene_name_short.columns = ['gene']
gene_name.index.names = ['index']
gene_name_short.index.names = ['index']

In [ ]:
intersection = list(set(gene_name_short['gene']).intersection(set(gene_name['gene'])).intersection(set(copy_number_df.columns[1:])) )

In [ ]:
###get common genes bewteen the 8 CCLE pools and custom pool
common_gene,_, ind_long = np.intersect1d(intersection, gene_name['gene'], return_indices = True)
_,_, ind_short = np.intersect1d(intersection, gene_name_short['gene'], return_indices = True)

common_gene_df = pd.DataFrame(common_gene, index = common_gene, columns = ['gene'])
common_gene_df.index.names = ['index']

In [ ]:
name_list = []
for name in meta.index[:-8]:
    obs = pd.read_csv(f"{name}_name.txt", sep="\t", header = None)
    name_list.append(obs[0])
unnest_name = list(chain.from_iterable(name_list))
name_df = pd.DataFrame(unnest_name, index = unnest_name, columns = ['Barcode'])
name_df.index.names = ['index']

In [ ]:
short_name_list = []
for name in meta.index[-8:]:
    obs = pd.read_csv(f"{name}_name.txt", sep="\t", header = None)
    short_name_list.append(obs[0])
unnest_name = list(chain.from_iterable(short_name_list))
short_name_df = pd.DataFrame(unnest_name, index = unnest_name, columns = ['Barcode'])
short_name_df.index.names = ['index']

In [ ]:
array_list = []
small_list = []
common_list = []

for d in df:
    if np.asarray(d).shape[0] != 28767:
        
        clean = np.asarray(d)
        clean = clean[ind_short,:]
        common_list.append(clean)
        #small_list.append(np.asarray(d))
        
    else:
        
        clean = np.asarray(d)
        clean = clean[ind_long,:]
        common_list.append(clean)
        #array_list.append(np.asarray(d))

In [ ]:
common_stack = np.hstack(common_list)

In [ ]:
merge_cellline = name_df.append(short_name_df)
sparse_X = sparse.csr_matrix(common_stack.T) 

In [ ]:
merge_adata = AnnData(sparse_X)
merge_adata.var = common_gene_df
merge_adata.obs = merge_cellline

In [ ]:
###Major file for downstream analysis

merge_adata.write_h5ad("ccle_merge_cpm.h5ad")
merge_adata.obs.to_csv("/data/ccle_obs.csv")

# merge_adata = sc.read_h5ad('ccle_merge_cpm.h5ad')
# obs = pd.read_csv("ccle_obs.csv")

### Process UMI count file
Match cell lines to CPM data processed above

In [1]:
umi_count = pd.read_csv("UMIcount_data.txt", sep= "\t")
umi_t = umi_count.T
umi_clean = umi_t.iloc[1:,2:]

In [37]:
obs = pd.read_csv("/data/ccle_obs.csv")
obs.index = obs['index']
obs = obs.drop(columns = 'index')

In [7]:
all_cell = sc.read_h5ad("ccle_merge_cpm.h5ad")

In [66]:
new_name = [str.replace(n,'.','-') for n in all_cell.obs.Barcode]

In [91]:
new_name=[]
for n in all_cell.obs.Barcode:
    if n[0]=='X':
        n=n[1:]
    new_name.append(str.replace(n,'.','-'))

In [88]:
umi_name = []

for a,b in zip(umi_count.iloc[0,1:], umi_count.columns[1:]): 
    if len(re.findall( "UPPER_AERODIGESTIVE_TRACT",a))!=0:
        b = str.replace(b,'c','')
    if a[0]=='X':
        a = a[1:]
    umi_name.append(a+"-"+b)

In [105]:
match_location = [np.where(np.asarray(umi_name) == n)[0][0] for n in new_name]
match_umi = umi_clean.iloc[match_location,:]
match_umi.columns = umi_count.iloc[:,0][2:]
array = np.asarray(match_umi).astype(float)
sparse_X = sparse.csr_matrix(array)

In [129]:
umi_adata = AnnData(sparse_X)
umi_adata.obs = obs
umi_adata.var = pd.DataFrame(umi_count.iloc[:,0][2:])

In [130]:
umi_adata

AnnData object with n_obs × n_vars = 53513 × 30314
    obs: 'Barcode', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'CCLE_ID', 'index.1', 'cancer_type', 'pool_id', 'n_cells', 'cancer_type_trunc'
    var: 'Unnamed: 0'

In [132]:
###final umi file for analysis 
umi_adata.write_h5ad("ccle_umi.h5ad")

Copying file://ccle_umi.h5ad [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][  1.5 GiB/  1.5 GiB]   83.7 MiB/s                                   
Operation completed over 1 objects/1.5 GiB.                                      


CompletedProcess(args='gsutil cp ccle_umi.h5ad gs://maus-cellranger/', returncode=0)

### Further processing (not used for analysis)

In [133]:
umi_adata.layers['umi'] = umi_adata.X

In [134]:
sc.pp.normalize_total(umi_adata, target_sum=1e4)
sc.pp.log1p(umi_adata)
sc.pp.highly_variable_genes(umi_adata, min_mean=0.01, max_mean=3, min_disp=0.2)

In [145]:
umi_adata.var.index = umi_adata.var['Unnamed: 0']
umi_adata.var.index.names=["index"]

In [146]:
umi_adata.var

,Unnamed: 0,highly_variable,means,dispersions,dispersions_norm
index,,,,,
A1BG,A1BG,True,0.226660,0.179397,0.896574
A1BG-AS1,A1BG-AS1,False,0.032659,-0.244404,-0.034471
A1CF,A1CF,False,0.007306,-0.058599,0.373721
A2M,A2M,True,0.035917,0.849223,2.368109
A2M-AS1,A2M-AS1,False,0.002309,-0.353224,-0.273538
...,...,...,...,...,...
SPATA13,SPATA13,False,0.068897,-0.154249,0.163589
TBC1D26,TBC1D26,False,0.000039,-0.621727,-0.863410
TIMM10B,TIMM10B,False,0.262999,-0.227587,-0.577479


In [147]:
umi_adata.var.loc[['XIST','RPS4Y1','RPS4Y2'],'highly_variable'] = False

In [148]:
umi_adata.var[umi_adata.var['Unnamed: 0'].isin(['XIST','RPS4Y1','RPS4Y2'])]

,Unnamed: 0,highly_variable,means,dispersions,dispersions_norm
index,,,,,
RPS4Y1,RPS4Y1,False,0.869183,2.230466,3.572005
RPS4Y2,RPS4Y2,False,0.001376,-0.693808,-1.021764
XIST,XIST,False,0.231898,1.067043,2.846636


In [149]:
np.sum(umi_adata.var.highly_variable)

3852

In [150]:
umi_adata.raw = umi_adata
umi_adata = umi_adata[:,umi_adata.var['highly_variable']]

In [154]:
Counter(umi_adata.obs['CCLE_ID'])

Counter({'NCIH2126_LUNG': 264,
         'SW579_THYROID': 217,
         'C32_SKIN': 554,
         'NCIH446_LUNG': 196,
         'HEC251_ENDOMETRIUM': 107,
         'MFE319_ENDOMETRIUM': 191,
         'SKNAS_AUTONOMIC_GANGLIA': 226,
         'NCIH2452_PLEURA': 538,
         'COLO741_SKIN': 329,
         'WM88_SKIN': 242,
         'JHH7_LIVER': 198,
         'KNS42_CENTRAL_NERVOUS_SYSTEM': 172,
         'MCF7_BREAST': 143,
         'HT1197_URINARY_TRACT': 74,
         'SNU899_UPPER_AERODIGESTIVE_TRACT': 83,
         'HCC38_BREAST': 200,
         'HEC108_ENDOMETRIUM': 172,
         'HT1376_URINARY_TRACT': 120,
         'SNU308_BILIARY_TRACT': 94,
         'TYKNU_OVARY': 117,
         'SW1271_LUNG': 115,
         'TM31_CENTRAL_NERVOUS_SYSTEM': 116,
         'NCIH747_LARGE_INTESTINE': 80,
         'KPNSI9S_AUTONOMIC_GANGLIA': 129,
         'SQ1_LUNG': 238,
         'CAOV3_OVARY': 221,
         'IALM_LUNG': 322,
         'BT474_BREAST': 125,
         'DKMG_CENTRAL_NERVOUS_SYSTEM': 271,
      

In [170]:
umi_adata.write_h5ad("ccle_umi_process.h5ad")

Copying file://ccle_umi_process.h5ad [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1 files][  2.2 GiB/  2.2 GiB]   90.2 MiB/s                                   
Operation completed over 1 objects/2.2 GiB.                                      


CompletedProcess(args='gsutil cp ccle_umi_process.h5ad gs://maus-cellranger/', returncode=0)